In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import anndata as ad

In [ ]:
def z_score_normalization(data):
    return (data - np.mean(data, axis=0)) / np.std(data, axis=0)

def reorder_columns(df, col_changed, col_position) -> pd.DataFrame:
    """
    Reordering colums. The second input can either be an integer for index or it can be a reference column name. If reference column name is chosen, the column will be placed after the reference column.
    """
    if not isinstance(df, pd.DataFrame):
        raise ValueError("The first argument must be a pandas DataFrame")
    if not isinstance(col_changed, str):
        raise ValueError("The second argument must be a string representing a column name")
    if not isinstance(col_position, (str, int)):
        raise ValueError("The third argument must be either a string representing a column name or an integer representing a column index")
    if isinstance(col_position, int):
        cols = df.columns.tolist()
        if col_changed in cols and col_position <= len(cols) + 1:
            cols.remove(col_changed)
        
            index = col_position
            cols.insert(index, col_changed)
            
            df = df[cols]
    else:
        cols = df.columns.tolist()
        if col_changed in cols and col_position in cols:
            cols.remove(col_changed)
            index = cols.index(col_position)
            cols.insert(index+1, col_changed)
            
            df = df[cols]
    return df

In [ ]:
df = pd.read_csv('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets2/breast_fibro_IMC/quantification/breast_fibro_IMC.csv')

In [ ]:
df[['celltype', 'cell_subtype']]

In [ ]:
df.columns

In [ ]:
df.FSP1

In [ ]:
df.drop(columns=['Panel', 'acID', 'Compartment', 'ImageInfo', 'tumour_nontumour'], inplace=True) 
df.rename(columns={'cell_subtype2': 'cell_type', 'Center_X': 'x', 'Center_Y': 'y'}, inplace=True)

In [ ]:
df.columns

In [ ]:
df = df.iloc[:, 15:].join(df.iloc[:, :15])

In [ ]:
df

In [ ]:
df.to_csv('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets2/breast_fibro_IMC/quantification/processed/breast_fibro_IMC_cleaned.csv', index=False)
df.to_csv('/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/breast_fibro_IMC_quantification.csv', index=False)

In [ ]:
df.drop(columns=['Histone H3', 'Iridium191', 'Iridium193'], inplace=True)

In [ ]:
df = pd.read_csv('/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/breast_fibro_IMC_quantification.csv')

In [ ]:
df.columns

In [ ]:
X_columns = df.columns[:df.columns.get_loc('ImageNumber')]
obs_columns =df.columns[df.columns.get_loc('ImageNumber'):]
adata = ad.AnnData(
    X=df[X_columns],
    obs=df[obs_columns],
    var=pd.DataFrame(index=X_columns)
)

In [ ]:

adata.raw = adata.copy()
adata.X = np.arcsinh(adata.X/1)
adata.layers['zscore'] = z_score_normalization(adata.X)
adata.obs['cell_type'] = adata.obs['cell_type'].astype('category')


In [ ]:
sc.pp.neighbors(add)
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color='cell_type')

In [ ]:
sc.pl.matrixplot(adata, var_names=adata.var_names, groupby='cell_type', cmap='vlag', dendrogram=True, use_raw=False, standard_scale='var')

Hypoxic and normal should most likely be epithelial cells. Lymphatic vessel quite low PDPN, We should think about binning the T cell subsets

## Harmonize celltype labels

In [ ]:
df = pd.read_csv('/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/breast_fibro_IMC_quantification.csv')
df

In [ ]:
df['cell_type'] = df['cell_type'].replace({'normal': 'Epithelial', 'hypoxic': 'hypoxic_Epithelial', 'SMA_Fibro': 'SMA+_Fibroblast', 'CD4': 'CD4+_T_cell',
                                             'CD8': 'CD8+_T_cell', 'Bcell': 'B_cell', 'T cell': 'T_cell', 'other': 'undefined'})




In [ ]:
df['cell_subtype'] = df['cell_subtype'].replace({'SMA_Fibro': 'SMA+_Fibroblast','Bcell': 'B_cell', 'Tcell': 'T_cell', 'other': 'undefined'})

In [ ]:
df['celltype'] = df['celltype'].replace({'Tumour':'Cancer', 'other': 'undefined'})

In [ ]:
df.to_csv('/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/breast_fibro_IMC_quantification.csv', index=False)

# Implement different levels of granularity

In [ ]:
df = pd.read_csv('/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/breast_fibro_IMC_quantification.csv')
df.columns

In [ ]:
df.BatchID.nunique(), df.SampleID.nunique(), df.RoiID.nunique()

In [ ]:
df.dtypes

In [ ]:
df[['BatchID', 'SampleID', 'RoiID', 'CellID', 'CellNumber', 'ImageNumber']]

In [ ]:
df['cell_type'].value_counts()

In [ ]:
df['level_2_cell_type'] = df['cell_type']
df['level_2_cell_type'] = df['level_2_cell_type'].replace({'SMA+_Fibroblast': 'Fibroblast', 'CD4+_T_cell': 'Lymphoid_immune','Lymphatic':'Vascular', 'CD8+_T_cell': 'Lymphoid_immune',
                                                           'Myeloid':'Myeloid_immune', 'Macrophage':'Myeloid_immune', 'FN_Cdh11_mCAF':'Fibroblast', 'Blood':'Vascular','Neutrophil':'Myeloid_immune',
                                                           'vCAF':'Fibroblast', 'B_cell':'Lymphoid_immune', 'CD10_CAF':'Fibroblast', 'CD4_Treg':'Lymphoid_immune', 'CD8_Treg':'Lymphoid_immune',
                                                           'T_cell':'Lymphoid_immune', 'HEV':'Vascular', 'IDO_CD4':'Lymphoid_immune', 'ki67_CD4':'Lymphoid_immune', 'CXCL13_CAF':'Fibroblast', 'IDO_CD8':'Lymphoid_immune',
                                                           'CD34_CAF':'Fibroblast', 'CD4_CXCL13':'Lymphoid_immune', 'CD73_CAF':'Fibroblast', 'ki67_CD8':'Lymphoid_immune', 'CD8_CXCL13':'Lymphoid_immune','IDO_CAF':'Fibroblast',
                                                            'CCL21_CAF':'Fibroblast', 'CA9_CD10_CAF':'Fibroblast'})
df['level_2_cell_type'].value_counts()

In [ ]:
df['level_1_cell_type'] = df['level_2_cell_type']
df['level_1_cell_type'] = df['level_1_cell_type'].replace({'Fibroblast':'Stromal', 'Lymphoid_immune':'Immune', 'Vascular':'Stromal', 'hypoxic Cancer':'Cancer', 'Vimentin high':'Stromal',
                                                            'Myeloid_immune':'Immune', 'Vascular':'Stromal'})
df['level_1_cell_type'].value_counts()

In [ ]:
df = reorder_columns(df, 'cell_type', 'level_1_cell_type')
df = reorder_columns(df, 'level_2_cell_type', 'level_1_cell_type')
df

In [ ]:
df.to_csv('/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/breast_fibro_IMC_quantification.csv', index=False)

# Encode uninfomrative variables

In [9]:
df = pd.read_csv('/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/breast_fibro_IMC_quantification.csv')
df.columns

Index(['FSP1', 'aSMA', 'Histone H3', 'CD11b_1', 'HLA-DR', 'CD146',
       'Cadherin-11', 'FAP', 'CD11b', 'VCAM1', 'CD20', 'CD68', 'IDO', 'CD3',
       'Podoplanin', 'CD11c', 'Carbonic Anhydrase IX', 'CD73', 'MMP9',
       'p75 (CD271)', 'CD10', 'Vimentin', 'FOXP3', 'CXCL13', 'PNAd', 'CD8',
       'Fibronectin', 'LYVE-1', 'PDGFR-b', 'CD34', 'CD4', 'vWF', 'CXCL-12',
       'CCL21', 'Keratin Epithelial', 'Iridium191', 'Iridium193', 'Ki-67',
       'CD45', 'Myeloperoxidase MPO', 'sample_id', 'cell_id', 'x', 'y', 'Area',
       'MajorAxisLength', 'MinorAxisLength', 'Area_Description', 'PatientID',
       'RoiID', 'unique_cell_id', 'celltype', 'cell_subtype',
       'level_1_cell_type', 'level_2_cell_type', 'cell_type'],
      dtype='object')

In [ ]:
df[['ImageNumber', 'CellNumber','BatchID', 'SampleID', 'RoiID', 'CellID']]

In [ ]:
df = df.rename(columns={'ImageNumber': 'sample_id', 'CellNumber': 'cell_id', 'SampleID': 'PatientID', 'CellID': 'unique_cell_id'})
df = df.drop(columns=['BatchID'])

In [ ]:
df.columns

In [ ]:
df.to_csv('/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/breast_fibro_IMC_quantification.csv', index=False)

# Bin some uninormative celtypes

In [14]:
df['cell_type'].value_counts()

cell_type
Cancer             160295
undefined           65482
hypoxic Cancer      62023
Vimentin high       61102
SMA+_Fibroblast     38984
CD4+_T_cell         24076
Lymphatic           16129
CD8+_T_cell         13643
Myeloid             12186
Macrophage          11912
FN_Cdh11_mCAF       10727
Blood               10531
Neutrophil           9933
vCAF                 9513
B_cell               8528
Fibroblast           4877
CD10_CAF             4653
CD4_Treg             4615
CD8_Treg             3907
T_cell               3819
HEV                  2975
IDO_CD4              2676
ki67_CD4             1631
CXCL13_CAF           1551
IDO_CD8              1198
CD34_CAF             1190
CD4_CXCL13            972
CD73_CAF              893
ki67_CD8              853
CD8_CXCL13            779
IDO_CAF               527
hypoxia               359
CCL21_CAF             323
CA9_CD10_CAF          259
Name: count, dtype: int64

In [15]:
df['level_2_cell_type'] = df['level_2_cell_type'].replace({'hypoxia': 'undefined'})
df['level_1_cell_type'] = df['level_1_cell_type'].replace({'hypoxia': 'undefined'})
df['cell_type'] = df['cell_type'].replace({'hypoxia': 'undefined'})

In [17]:
df.to_csv('/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/breast_fibro_IMC_quantification.csv', index=False)